<a href="https://colab.research.google.com/github/BhuvanKalyanGV/Web_scapping/blob/main/OmiClear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openpyxl.styles import Font
import re
from datetime import datetime, timedelta

# Send a GET request to the website
url = "https://www.omiclear.pt/en/dados-mercado?date=2024-05-23&product=NG&zone=ES&instrument=FGF"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the input tag with the date value
input_tag = soup.find('input', {'type': 'date'})

# Extract the date value from the input tag
date_value = input_tag['value']

# Find all tables on the page
tables = soup.find_all('table')

# Create a list to store the table data
table_data = []

# Loop through each table
for table in tables:
    # Extract the table headers
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]

    # Extract the table data
    data = []
    for row in table.find_all('tr')[1:]:
        row_data = []
        for td in row.find_all('td'):
            # Extract the text outside the <span> tags
            text = td.text.strip()
            for span in td.find_all('span'):
                text = text.replace(span.text, '')
            row_data.append(text.strip())
        data.append(row_data)

    # Create a Pandas DataFrame from the table data
    # Use the first row as the column names
    df = pd.DataFrame(data[1:], columns=data[0])

    # Add the DataFrame to the list
    table_data.append(df)

# Concatenate the DataFrames
combined_df = pd.concat(table_data, ignore_index=True)

# Add start and end date columns to the DataFrame
combined_df['Start Date'] = None
combined_df['End Date'] = None

def calculate_week_dates(week_number, year):
    # Calculate the first day of the year
    first_day = datetime(year, 1, 1)

    # Calculate the first Monday of the year
    first_monday = first_day + timedelta(days=7 - first_day.weekday())

    # Calculate the start date of the given week
    start_date = first_monday + timedelta(days=(week_number - 1) * 7)
    start_date = start_date - timedelta(days=1)

    # Calculate the end date of the given week
    end_date = start_date + timedelta(days=6)

    return start_date, end_date

# Define a function to extract start and end dates from contract name
def extract_dates(contract_name):
    start_date = None
    end_date = None
    if 'WkDs' in contract_name:
        match = re.search(r'WkDs(\d{2})-(\d{2})', contract_name)
        if match:
            week_num = int(match.group(1))
            year = int('20' + match.group(2))
            start_date, end_date = calculate_week_dates(week_num, year)
            start_date = start_date.strftime('%d%b-%y')
            end_date = end_date.strftime('%d%b-%y')
    elif 'D' in contract_name:
        # Day-based contract
        match = re.search(r'(\d{1,2}[A-Z][a-z]{2}-\d{2})', contract_name)
        if match:
            date_str = match.group(1)
            start_date = date_str
            end_date = date_str
    elif 'WE' in contract_name:
        # Weekend-based contract
        match = re.search(r'(\d{1,2}[A-Z][a-z]{2}-\d{2})', contract_name)
        if match:
            date_str = match.group(1)
            start_date = date_str
            end_date = increment_date(date_str, 1)
    elif 'BoM' in contract_name:
        # Balance of month-based contract
        match = re.search(r'(\d{1,2}[A-Z][a-z]{2}-\d{2})', contract_name)
        if match:
            date_str = match.group(1)
            start_date = date_str
            end_date = last_day_of_month(start_date)
    elif 'M' in contract_name:
        # Month-based contract
        match = re.search(r'([A-Z][a-z]{2})', contract_name)
        if match:
            month_str = match.group(1)
            start_date = f'01{month_str}-{contract_name[-2:]}'
            end_date = last_day_of_month(start_date)
    elif 'Q' in contract_name:
        # Quarter-based contract
        quarter = int(re.search(r'Q(\d)', contract_name).group(1))
        year = int(contract_name[-2:])
        if quarter == 1:
            start_date = f'01Jan-{year}'
            end_date = f'31Mar-{year}'
        elif quarter == 2:
            start_date = f'01Apr-{year}'
            end_date = f'30Jun-{year}'
        elif quarter == 3:
            start_date = f'01Jul-{year}'
            end_date = f'30Sep-{year}'
        elif quarter == 4:
            start_date = f'01Oct-{year}'
            end_date = f'31Dec-{year}'
    elif 'Win' in contract_name or 'Sum' in contract_name:
        # Season-based contract
        if 'Win' in contract_name:
            start_date = f'01Jul-{contract_name[-2:]}'
            end_date = f'31Dec-{contract_name[-2:]}'
        else:
            start_date = f'01Jan-{contract_name[-2:]}'
            end_date = f'30Jun-{contract_name[-2:]}'
    elif 'YR' in contract_name:
        # Year-based contract
        year = int(contract_name[-2:])
        start_date = f'01Jan-{year}'
        end_date = f'31Dec-{year}'

    return start_date, end_date


# Define helper functions
def increment_date(date_str, days):
    # Increment date by given number of days
    date_obj = pd.to_datetime(date_str, format='%d%b-%y')
    date_obj += pd.Timedelta(days=days)
    return date_obj.strftime('%d%b-%y')

def decrement_date(date_str, days):
    # Decrement date by given number of days
    date_obj = pd.to_datetime(date_str, format='%Y-%m-%d')
    date_obj -= pd.Timedelta(days=days)
    return date_obj.strftime('%Y-%m-%d')

def last_day_of_month(date_str):
    # Get last day of month
    date_obj = pd.to_datetime(date_str, format='%d%b-%y')
    last_day = pd.Period(date_obj, freq='M').end_time
    return last_day.strftime('%d%b-%y')

# Iterate over the rows and set the start and end dates
for index, row in combined_df.iterrows():
    contract_name = row['Contract name']
    start_date, end_date = extract_dates(contract_name)
    combined_df.loc[index, 'Start Date'] = start_date
    combined_df.loc[index, 'End Date'] = end_date

# Create an Excel writer
with pd.ExcelWriter(f'{date_value}.xlsx') as writer:
    # Write the combined DataFrame to a single sheet
    combined_df.to_excel(writer, sheet_name='FGF', index=False, header=True)

    # Get the workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['FGF']

    # Set the format for the header row
    for cell in worksheet[1]:
        cell.font = Font(bold=True)

print("Data saved to", date_value, ".xlsx")